In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('lib')
from lib.read_data import *
import time
import torch
import torch.functional as F
from model import *
from sm_model import *
torch.set_default_dtype(torch.float64)

In [ ]:
N = 128
DIM = 3
outdir = f"output/output_{DIM}D_{N}/checkpt_mixedBCs_M11_ritz1600_rhs800_l5_trilinear.tar"
checkpt = torch.load(outdir)

In [ ]:
train_loss = checkpt['training_loss']
valid_loss = checkpt['validation_loss']
fig, axes = plt.subplots(2)
axes[0].plot(train_loss, label='Training')
axes[1].plot(valid_loss, label='Validation', c='orange')
axes[0].legend()
axes[1].legend()
plt.savefig("train_loss.png")
plt.show()